In [32]:
# imported dependencies
import pennylane as qml
from itertools import product
import numpy as np

In [57]:
from pennylane.pauli import pauli_group
n_qubits = 2
for p in pauli_group(n_qubits):

    matrix = qml.matrix(p)
    identityMatrix = -np.identity(matrix.shape[0])
    # isNegative = np.allclose(matrix,identityMatrix)
    # if isNegative:
    #     print("wow")
    #print(qml.matrix(p))
    print(p)


Identity(wires=[0])
PauliZ(wires=[1])
PauliZ(wires=[0])
PauliZ(wires=[0]) @ PauliZ(wires=[1])
PauliX(wires=[1])
PauliY(wires=[1])
PauliZ(wires=[0]) @ PauliX(wires=[1])
PauliZ(wires=[0]) @ PauliY(wires=[1])
PauliX(wires=[0])
PauliX(wires=[0]) @ PauliZ(wires=[1])
PauliY(wires=[0])
PauliY(wires=[0]) @ PauliZ(wires=[1])
PauliX(wires=[0]) @ PauliX(wires=[1])
PauliX(wires=[0]) @ PauliY(wires=[1])
PauliY(wires=[0]) @ PauliX(wires=[1])
PauliY(wires=[0]) @ PauliY(wires=[1])


In [227]:
## 2-qubit Pauli Group
# Encode 1 logical qubit into 2 physical qubits
S1 = qml.PauliX(wires=[0]) @ qml.PauliZ(wires=[1])
S2 = qml.PauliZ(wires=[1]) @ qml.Identity(wires=[0])
qml.is_commuting(S1,S2)
# Stabilizer group aka bodyguard
S = [qml.matrix(S1),qml.matrix(S2)]
P = 1/2 * (S[0] + S[1])
#P = qml.Hermitian(P,wires=[0,1])
G1 = S1


In [235]:
dev = qml.device("default.qubit", wires=2, shots=1)
@qml.qnode(dev)
def create_entangled_state(wires):
    #qml.Hadamard(wires=wires[0])
    qml.PauliX(wires=wires[0])
    qml.PauliY(wires=wires[0])
    qml.PauliZ(wires=wires[0])
    return qml.state()

@qml.qnode(dev)
def hadamard_test(wires):
    qml.Hadamard(wires=wires[1])
    qml.ctrl(G1,control=wires[1])(wires=wires[0])
    return qml.measure(1)

create_entangled_state(wires=[0,1])
hadamard_test(wires=[0,1])

#p = create_entangled_state([0])
#num = np.matmul(np.matmul(P,p),P)
# den = np.sum(np.matmul(p,P))
# pDet = num/den
# print(p)
# print(pDet)


QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.

In [192]:
dev = qml.device("default.qubit", wires=2, shots=1)
@qml.qnode(dev)
def fig1(wires):
    qml.BasisStatePreparation(p,wires=[0,1])
    qml.Hadamard(wires=wires[0])
    qml.ctrl(G1,wires=wires[0])

    return qml.measure()

[[ 0 -1]
 [ 1  0]]


C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pennylane\operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
